In [1]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2

In [2]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# lang models dir
llm_path = f'{data_dir}Language Models'

# sites
site_list=['Indeed', 'Glassdoor', 'LinkedIn']

# columns
cols=['Sector', 
      'Sector Code', 
      'Gender', 
      'Age', 
      'Language', 
      'Dutch Requirement', 
      'English Requirement', 
      'Gender_Female', 
      'Gender_Mixed', 
      'Gender_Male', 
      'Age_Older', 
      'Age_Mixed', 
      'Age_Younger', 
      'Gender_Num', 
      'Age_Num', 
      '% Female', 
      '% Male', 
      '% Older', 
      '% Younger']

int_variable: str = 'Job ID'
str_variable: str = 'Job Description'
gender: str = 'Gender'
age: str = 'Age'
language: str = 'en'
str_cols = ['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Company URL', 'Job URL', 'Tracking ID']
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'


In [3]:
import string
import re
import time
import json
import csv
import glob
import pickle
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag, pos_tag_sents
from spacy.pipeline import Sentencizer


In [35]:
df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_dropped.pkl').reset_index(drop=True)


In [36]:
len(df_jobs)


62577

In [37]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
    lambda job_description: ' '.join(job_description.split('/')) if '/' in job_description else job_description
)


In [46]:
abb_dict = {
    r'incl\.': 'including', 
    r'e\.g\.': 'for example', 
    r'e\.g': 'for example', 
    r'etc\.': 'et cetera', 
}


In [47]:
str_fix_incl = 'Apply appropriate and effective communication methods to senior management and important stakeholders'
str_fix_eg = 'Partner with Procurement in order to manage suppliers for the projects & programs in scope'



In [49]:
df_jobs['Job Description'] = df_jobs['Job Description'].replace(abb_dict, regex=True)

In [50]:
df_jobs['Job Description'][df_jobs['Job Description'].str.contains(str_fix_incl)]


35      Purpose\n\nAs a Studio A Director\n\nSpecial P...
3423    Purpose:\nAs a Manager Digital Strategy and Pr...
Name: Job Description, dtype: object

In [51]:
re.findall(str_fix_eg, df_jobs['Job Description'][35])

['Partner with Procurement in order to manage suppliers for the projects & programs in scope']

In [52]:
list(re.finditer(str_fix_eg, df_jobs['Job Description'][35]))

[<re.Match object; span=(1461, 1551), match='Partner with Procurement in order to manage suppl>]

In [54]:
df_jobs['Job Description'][35][1461:1580]

'Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, sta'

In [31]:
# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('incl.', 'including')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g.', 'e.g')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('e.g', 'for example')
# )

# df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].apply(
#     lambda job_description: job_description.replace('etc.', 'et cetera')
# )


In [55]:
# df_jobs = df_jobs[:100]

In [56]:
# df_jobs = df_jobs.sample(n=100, random_state=1)

In [57]:
# df_jobs = pd.read_pickle(f'{data_dir}df_jobs_raw_glob_paths_10.pkl').reset_index(drop=True)


In [58]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62577 entries, 0 to 62576
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     62577 non-null  object 
 1   Platform           62577 non-null  object 
 2   Job ID             62577 non-null  object 
 3   Job Title          62577 non-null  object 
 4   Company Name       62574 non-null  object 
 5   Location           62577 non-null  object 
 6   Job Description    62577 non-null  object 
 7   Rating             3975 non-null   float64
 8   Employment Type    61995 non-null  object 
 9   Company URL        59263 non-null  object 
 10  Job URL            62577 non-null  object 
 11  Job Age            62577 non-null  object 
 12  Job Age Number     62577 non-null  object 
 13  Collection Date    62577 non-null  object 
 14  Data Row           58599 non-null  float64
 15  Tracking ID        58599 non-null  object 
 16  Industry           591

In [59]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name',
       'Location', 'Job Description', 'Rating', 'Employment Type',
       'Company URL', 'Job URL', 'Job Age', 'Job Age Number',
       'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date',
       'Type of ownership'],
      dtype='object')

In [60]:
len(df_jobs.loc[(df_jobs['Job Description'].str.contains(str_fix_incl)) | (df_jobs['Job Description'].str.contains(str_fix_eg))])


2

In [61]:
# job_descriptions = list(set(df_jobs['Job Description'].to_list()))


In [62]:
job_descriptions = list(
    set(
        df_jobs['Job Description'].loc[
            (df_jobs['Job Description'].str.contains(str_fix_incl)) |
            (df_jobs['Job Description'].str.contains(str_fix_eg))
        ].to_list()
    )
)


In [63]:
len(job_descriptions)


2

In [64]:
# Load NLK
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords

nltk_path = f'{llm_path}/nltk'
nltk.data.path.append(nltk_path)

nltk.download('words', download_dir = nltk_path)
nltk.download('stopwords', download_dir = nltk_path)
nltk.download('punkt', download_dir = nltk_path)
nltk.download('averaged_perceptron_tagger', download_dir = nltk_path)

stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)


[nltk_data] Downloading package words to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nyxinsane/Documents/Work - UvA/Automating
[nltk_data]     Equity/Study 1/Study1_Code/data/Language
[nltk_data]     Models/nltk...
[nltk_data]   Package averaged_perceptron_

In [65]:
job_descriptions[0][1500:2000]

'and important stakeholders throughout the project lifecycle.\nAs conflicts and escalations arise within projects, identify solutions and drive their resolution in a timely and appropriate manner.\nManage change within projects and ensure changes are smoothly and successfully implemented to achieve lasting benefits.\nProject Controlling\nManage project controlling as an independent element to ensure that actual project costs are in line the committed budget.\nConduct engagement reviews. Verify complia'

In [77]:
# pattern = r'[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?<=[a-z])(?=[A-Z])'
pattern = r'[\n]+|[,]{2,}|[|]{2,}|[\n\r]+|(?<=[a-z]\.)(?=\s*[A-Z])|(?=\:+[A-Z])'



In [78]:
# nltk_sentencizer = []

# for job_description in job_descriptions:
#     for sentence in sent_tokenize(job_description):
#         for sent in re.split(pattern, sentence):
#             nltk_sentencizer.append(sent)

In [79]:
nltk_sentencizer = [
    sent 
    for job_description in job_descriptions 
    for sentence in sent_tokenize(job_description) 
    for sent in re.split(pattern, sentence)
    if len(sent) != 0
]

In [80]:
len(nltk_sentencizer)

133

In [81]:
for idx, sent in enumerate(nltk_sentencizer):
    if str_fix_eg in sent:
        print(idx)

84


In [82]:
next(idx for idx, sent in enumerate(nltk_sentencizer) if str_fix_eg in sent)

84

In [83]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'

for idx, sent in enumerate(nltk_sentencizer):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

18
87


In [85]:
nltk_sentencizer[84:95]

['Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.',
 'Communication   stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.',
 'Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.',
 'Project Controlling',
 'Manage project controlling as an independent element to ensure that actual project costs are in line the committed budget.',
 'Lead or conduct engagement reviews.',
 'Verify implementation of quality assurance procedures.',
 'Ensure project & program delivery aga

In [86]:
# sentences split on e.g. and incl.
nltk_sentencizer[
    next(idx for idx, sent in enumerate(nltk_sentencizer) if str_fix_eg in sent):next(idx for idx, sent in enumerate(nltk_sentencizer) if str_fix_incl.split('\(')[0] in sent)+2
]


[]

In [87]:
for idx, sent in enumerate(nltk_sentencizer):
    if 'Power' in sent:
        print(idx)

62
129


In [88]:
next(idx for idx, sent in enumerate(nltk_sentencizer) if 'Power' in sent)

62

In [89]:
nltk_sentencizer[next(idx for idx, sent in enumerate(nltk_sentencizer) if 'Power' in sent)]

'Strong MS-Office skills (Word, Excel, PowerPoint)'

In [91]:
for idx, sentence in enumerate(nltk_sentencizer[84:95]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.

Sentence 2: Communication   stakeholder management

Sentence 3: Act as the key contact person for strategic stakeholders.

Sentence 4: Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.

Sentence 5: As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.

Sentence 6: Drive change management activities for respective projects & programs and ensure changes are smoothly and successfully implemented to achieve lasting benefits.

Sentence 7: Project Controlling

Sentence 8: Manage project controlling as an independent element to ensure that actual project costs are in line the committed budget.

Sentence 9: Lead or conduct engagement reviews.

Sentence 10: Veri

In [92]:
# nltk_tokenizer = []

# for job_sentence in nltk_sentencizer:
#     for token in word_tokenize(job_sentence):
#         if len(token) != 0 and token != '...' and token.lower() not in set(stopwords.words('english')) and token.lower() not in list(string.punctuation):
#             nltk_tokenizer.append(token)


In [93]:
nltk_tokenizer = [
    token 
    for job_sentence in nltk_sentencizer 
    for token in word_tokenize(job_sentence) 
    if len(token) != 0 
    and token != '...' 
    and not token.lower() in set(stopwords.words('english')) 
    and not token.lower() in list(string.punctuation) 
]

In [94]:
'...' in nltk_tokenizer

False

In [95]:
len(nltk_tokenizer)

915

In [96]:
nltk_token_tags = pos_tag(nltk_tokenizer)

In [145]:
nltk_token_tags[0]

('Purpose', 'NNP')

In [97]:
type(nltk_token_tags[0])

tuple

In [98]:
len(nltk_token_tags)

915

In [99]:
# Function to make a list of punctuations that determine sentence boundry, i.e., split characters
def make_custom_punct_chars(main_punct_chars = [':', '|'], repeated_punct_chars = ['\n', ',']):
    custom_punct_chars = []
    temp_multi = []
    temp_spaced = []

    for punct_char in main_punct_chars:
        custom_punct_chars+= f'{punct_char}', f'{punct_char} '

    for idx in range(4):
        for punct_char in repeated_punct_chars:
            temp_multi.append(f'{punct_char}'*int(idx+1))
            temp_spaced.append(f'{punct_char} '*int(idx+1))

    for multi, spaced in zip(temp_multi, temp_spaced):
        custom_punct_chars+= multi, spaced

    custom_punct_chars.remove(',')
    custom_punct_chars.remove(', ')

    return custom_punct_chars


In [100]:
custom_punct_chars = make_custom_punct_chars()

In [101]:
custom_punct_chars

[':',
 ': ',
 '|',
 '| ',
 '\n',
 '\n ',
 '\n\n',
 '\n \n ',
 ',,',
 ', , ',
 '\n\n\n',
 '\n \n \n ',
 ',,,',
 ', , , ',
 '\n\n\n\n',
 '\n \n \n \n ',
 ',,,,',
 ', , , , ']

In [102]:
# Set up Spacy
import spacy
from spacy.symbols import ORTH, LEMMA

nlp = spacy.load('en_core_web_sm')


In [103]:
sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)


In [104]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    with open(f'{data_dir}punctuations.txt', 'wb') as f:
        pickle.dump(sentencizer.punct_chars, f)

In [105]:
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_char = pickle.load(f)

In [106]:
# special_cases = {":)": [{"ORTH": ":)"}]}
# prefix_re = re.compile(r'''^[\\[\\("']''')
# suffix_re = re.compile(r'''[\\]\\)"']$''')
# infix_re = re.compile(r'''[-~]''')
# simple_url_re = re.compile(r'''^https?://''')

# def custom_tokenizer(nlp):
#     return Tokenizer(nlp.vocab, rules=special_cases,
#                                 prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 url_match=simple_url_re.match)

In [107]:
# nlp = spacy.load('en_core_web_sm')
# nlp.tokenizer.add_special_case('incl.', [{ORTH: 'incl', LEMMA: 'including'}])


In [108]:
# from spacy.symbols import ORTH, LEMMA

# special_cases_dict = {
#     'incl.': [{65: 'incl', 67: 'including'}],
#     'incl. ': [{65: 'incl', 67: 'including'}],
#     '(incl.': [{65: 'incl', 67: 'including'}],
#     'etc.': [{65: 'etc', 67: 'et cetera'}],
#     'etc. ': [{65: 'etc', 67: 'et cetera'}],
#     'e.g.': [{65: 'e.g', 67: 'for example'}],
#     'e.g. ': [{65: 'e.g', 67: 'for example'}],
# }

# nlp.tokenizer.rules.update(special_cases_dict)


In [109]:
# nlp.tokenizer.rules['e.g.']

In [110]:
# # Spacy sentencizer
# spacy_sentencizer = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_description in job_descriptions:
#         for sentence in nlp(job_description).sents:
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencizer.append(sent)

In [111]:
# Spacy sentencizer
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    spacy_sentencizer = [
        sent 
        for job_description in job_descriptions 
        for sentence in nlp(job_description).sents 
        for sent in re.split(pattern, sentence.text) 
        if len(sent) != 0 
    ]

In [112]:
# # Spacy tokenizer
# spacy_tokenizer = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_sentence in spacy_sentencizer:
#         for token in nlp.tokenizer(job_sentence):
#             if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
#                 spacy_tokenizer.append(token.text)

In [113]:
type(spacy_sentencizer[0])

str

In [114]:
len(spacy_sentencizer)

135

In [115]:
# Spacy tokenizer
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    spacy_tokenizer = [
        token.text 
        for job_sentence in spacy_sentencizer 
        for token in nlp.tokenizer(job_sentence) 
        if len(token) != 0 
        and not token.is_stop 
        and not token.is_punct 
        and not token.text in custom_punct_chars
    ]

In [116]:
type(spacy_tokenizer[0])

str

In [117]:
len(spacy_tokenizer)

910

In [118]:
for idx, sent in enumerate(spacy_sentencizer):
    if str_fix_eg in sent:
        print(idx)

86


In [119]:
next(idx for idx, sent in enumerate(spacy_sentencizer) if str_fix_eg in sent)

86

In [120]:
# str_fix = 'Apply appropriate and effective communication methods to senior management and important stakeholders \(incl.'


for idx, sent in enumerate(spacy_sentencizer):
    if str_fix_incl.split('\(')[0] in sent:
        print(idx)

18
89


In [121]:
next(idx for idx, sent in enumerate(spacy_sentencizer) if str_fix_incl.split('\(')[0] in sent)

18

In [122]:
spacy_sentencizer[86:91]

['Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.',
 'Communication   stakeholder management',
 'Act as the key contact person for strategic stakeholders.',
 'Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.',
 'As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.']

In [123]:
for idx, sent in enumerate(spacy_sentencizer):
    if 'Power' in sent:
        print(idx)

63
131


In [124]:
next(idx for idx, sent in enumerate(spacy_sentencizer) if 'Power' in sent)

63

In [125]:
spacy_sentencizer[63]

'Strong MS-Office skills (Word, Excel, PowerPoint)'

In [126]:
for idx, sentence in enumerate(spacy_sentencizer[86:91]):
    print(f'Sentence {idx+1}: {sentence}\n')

Sentence 1: Partner with Procurement in order to manage suppliers for the projects & programs in scope, for example interviews, staffing, supplier costs.

Sentence 2: Communication   stakeholder management

Sentence 3: Act as the key contact person for strategic stakeholders.

Sentence 4: Apply appropriate and effective communication methods to senior management and important stakeholders (including vendor) throughout the project lifecycle.

Sentence 5: As conflicts and escalations arise within projects, identify solutions and manage the resolution in a timely and appropriate manner.



In [127]:
# # Spacy tokenizer
# spacy_tokenizer = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_sentence in spacy_sentencizer:
# #         doc = nlp.tokenizer(job_sentence)
#         spacy_tokenizer.extend(
#             [
#                 token.text for token in nlp.tokenizer(job_sentence) 
#                 if token.text not in custom_punct_chars
#                 and not token.is_stop 
                
#             ]
#         )

In [128]:
len(spacy_tokenizer)

910

In [129]:
spacy_tokenizer[:10]

['Purpose',
 'Manager',
 'Digital',
 'Strategy',
 'Programs',
 'adidas',
 'market',
 'ecom',
 'organization',
 'task']

In [158]:
# # Spacy sentencizer
# spacy_sentencizer = []
# nlp_token_tagging = []

# if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
#     for job_description in job_descriptions:
#         for token in nlp(job_description):
#             if len(token) != 0 and not token.is_stop and not token.is_punct and token.text not in custom_punct_chars:
#                 nlp_token_tagging.append(tuple([token.text, token.tag_]))
# #         for sentence in doc.sents:
# #             for sent in re.split(pattern, sentence.text):
# #                 if len(sent) != 0:
# #                     spacy_sentencizer.append(sent)

In [160]:
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    nlp_token_tagging = [
        tuple([token.text, token.tag_]) 
        for job_description in job_descriptions 
        for token in nlp(job_description) 
        if len(token) != 0 
        and not token.is_stop 
        and not token.is_punct 
        and not token.text in custom_punct_chars
    ]

In [161]:
nlp_token_tagging

[('Purpose', 'NN'),
 ('Manager', 'NNP'),
 ('Digital', 'NNP'),
 ('Strategy', 'NNP'),
 ('Programs', 'NNP'),
 ('adidas', 'NNP'),
 ('market', 'NNP'),
 ('ecom', 'NNP'),
 ('organization', 'NN'),
 ('task', 'NN'),
 ('manage', 'VB'),
 ('deliver', 'VB'),
 ('small', 'JJ'),
 ('digital', 'JJ'),
 ('projects', 'NNS'),
 ('significant', 'JJ'),
 ('RFCs', 'NNS'),
 ('sub', 'NNS'),
 ('elements', 'NNS'),
 ('mid', 'JJ'),
 ('scale', 'JJ'),
 ('digital', 'JJ'),
 ('projects', 'NNS'),
 ('project', 'NN'),
 ('scope', 'NN'),
 ('local', 'JJ'),
 ('market', 'NN'),
 ('level', 'NN'),
 ('brands', 'NNS'),
 ('functions', 'NNS'),
 ('objective', 'NN'),
 ('conduct', 'VB'),
 ('projects', 'NNS'),
 ('boundaries', 'NNS'),
 ('time', 'NN'),
 ('cost', 'NN'),
 ('quality', 'NN'),
 ('focusing', 'VBG'),
 ('consumers', 'NNS'),
 ('expectations', 'NNS'),
 ('requirements', 'NNS'),
 ('Planning', 'NN'),
 ('executing', 'VBG'),
 ('tracking', 'NN'),
 ('projects', 'NNS'),
 ('cooperation', 'NN'),
 ('respective', 'JJ'),
 ('functions', 'NNS'),
 ('pro

In [135]:
# Spacy sentencizer
# spacy_sentencizer = []

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    for job_description in job_descriptions:
        for token in nlp(job_description):
            print(token.text, token.tag_)
#             for sent in re.split(pattern, sentence.text):
#                 if len(sent) != 0:
#                     spacy_sentencizer.append(sent)

Purpose NOUN NN
: PUNCT :

 SPACE _SP
As ADP IN
a DET DT
Manager PROPN NNP
Digital PROPN NNP
Strategy PROPN NNP
and CCONJ CC
Programs PROPN NNP
in ADP IN
adidas PROPN NNP
market PROPN NNP
ecom PROPN NNP
organization NOUN NN
your PRON PRP$
task NOUN NN
is AUX VBZ
to PART TO
manage VERB VB
and CCONJ CC
deliver VERB VB
small ADJ JJ
digital ADJ JJ
projects NOUN NNS
, PUNCT ,
significant ADJ JJ
RFCs NOUN NNS
or CCONJ CC
sub NOUN NNS
- NOUN NNS
elements NOUN NNS
of ADP IN
mid ADJ JJ
- ADJ JJ
scale ADJ JJ
digital ADJ JJ
projects NOUN NNS
. PUNCT .
The DET DT
project NOUN NN
scope NOUN NN
is AUX VBZ
on ADP IN
a DET DT
local ADJ JJ
( PUNCT -LRB-
market NOUN NN
) PUNCT -RRB-
level NOUN NN
across ADP IN
brands NOUN NNS
and CCONJ CC
functions NOUN NNS
. PUNCT .
Your PRON PRP$
objective NOUN NN
is AUX VBZ
to PART TO
conduct VERB VB
the DET DT
projects NOUN NNS
within ADP IN
the DET DT
boundaries NOUN NNS
of ADP IN
time NOUN NN
, PUNCT ,
cost NOUN NN
& CCONJ CC
quality NOUN NN
while SCONJ IN
focusin

In [ ]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')

matcher = Matcher(nlp.vocab)

bigram_rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

rules = [
    ['VERB', 'ADJ', 'NOUN'],
    ['NOUN', 'VERB', 'ADV'],
    ['NOUN', 'ADP', 'NOUN'],
    # more rules here...
]

trigram_patterns = [[{"POS": i} for i in j] for j in rules]


In [ ]:
trigram_patterns

In [ ]:
patterns = {
    'noun_verb': [{'POS': 'NOUN'}, {'POS': 'VERB'}],
    'verb_noun': [{'POS': 'VERB'}, {'POS': 'NOUN'}],
    'adj_noun': [{'POS': 'ADJ'}, {'POS': 'NOUN'}],
    'adj_propn': [{'POS': 'ADJ'}, {'POS': 'PROPN'}],
}

In [ ]:
for pattern_name, pattern in patterns.items():
    matcher.add(pattern_name, [pattern])


In [ ]:
for job_description in job_descriptions:
    doc = nlp(job_description)
    matches = matcher(doc)

In [ ]:
for match_id, start, end in matches:
    
    # Get string representation
    string_id = nlp.vocab.strings[match_id]

    # The matched span
    span = doc[start:end]
    
    print(repr(span.text))
    print(match_id, string_id, start, end)
    print()

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [ ]:
# for sentence in spacy_sentencizer:
#     print(model.encode(sentence))



In [ ]:
from transformers.models.electra.modeling_tf_electra import TFElectraMainLayer

In [ ]:
from transformers import AutoTokenizer, AutoModelForPreTraining

tokenizer = AutoTokenizer.from_pretrained("pavanchhatpar/electra-base-sentence-splitter")

model = AutoModelForPreTraining.from_pretrained("pavanchhatpar/electra-base-sentence-splitter", from_tf=True)


In [ ]:
nlp = pipeline('sentence-splitter', model)